## Seif MA Dispatching

In [1]:
!pip install gspread_dataframe


In [2]:
!pip install oauth2client


In [9]:
progress_perc = round((Done / Tasks) * 100, 1)

print(f"percentage of taks done {progress_perc}%")

percentage of taks done 33.1%


In [11]:
    # test = ret_metabase('MOROCCO', 18879)
    # test2 = ret_metabase('MOROCCO', 10368)
    # test3 =ret_metabase('MOROCCO', 10728)
    
    # print("10410: \n", test)
    test
    # print("10368: \n", test2.columns.tolist())
    # print("10728: \n", test3.columns.tolist())
    print(data_assigned.columns.tolist())

[]


In [12]:
import pytz
from datetime import datetime
import pandas as pd
import numpy as np

morocco_tz = pytz.timezone('Africa/Casablanca')
now = datetime.now(morocco_tz)
current_time = now.time()
current_day = now.strftime('%A')
print(current_day)
hour = int(str(now.time())[0:2])

agents_df = google_sheets('MA daily_automatic_assignment', 'attendance_filtered', 'get')
# print(agents_df)
print(hour)

agents_df_copy = agents_df.copy() 
agents_df.columns = agents_df.columns.str.strip().str.lower()
agents_df_copy['assignment_start_time'] = agents_df['start_time'].astype(int)
agents_df_copy['assignment_end_time'] = agents_df['end_time'].astype(int)
agents_df_copy['current_day'] = agents_df['day'].astype(str).str.lower() == 'true'


# Fetch data for mapped retailer-agent 
mapping_df = google_sheets('MA Telesales Cycle Assignment', 'Sheet1', 'get')


agents_df_copy['assign_data'] =  np.where(
    (hour >= agents_df_copy['assignment_start_time'])
    & (hour <= agents_df_copy['assignment_end_time'])
    & (agents_df['day'] == True),
    'yes',
    'no' )

# store all available clocked-in agents for today
available_agents = agents_df_copy.loc[agents_df_copy['assign_data'] == 'yes']
available_agents_list = available_agents['agent_id'].values.tolist()
number_of_available_agents = len(available_agents_list)

Monday
/home/ec2-user/service_account_key.json
10
/home/ec2-user/service_account_key.json


In [14]:
# Retention & Reactivation Tasks for 1st time slot
Retention_dfs = [
    ret_metabase('MOROCCO', 18879),
    # ret_metabase('MOROCCO', "<query_id>"),
    # ret_metabase('MOROCCO', "<query_id>"),
    # ret_metabase('MOROCCO', "<query_id>"),
    # ret_metabase('MOROCCO', "<query_id>")
]

for df in Retention_dfs:
    df.columns = map(str.lower, df.columns)

# Marketing Tasks for 2nd time slot
Marketing_dfs = [
    ret_metabase('MOROCCO', 10368),
    # ret_metabase('MOROCCO', "<query_id>"),
    # ret_metabase('MOROCCO', "<query_id>"),
    # ret_metabase('MOROCCO', "<query_id>"),
    # ret_metabase('MOROCCO', "<query_id>")
]

for df in Marketing_dfs:
    df.columns = map(str.lower, df.columns)

# Marketing & E-commerce  Tasks for 3rd time slot 
Ecom_dfs = [
    ret_metabase('MOROCCO', 10728),
    # ret_metabase('MOROCCO', "<query_id>"),
    # ret_metabase('MOROCCO', "<query_id>"),
    # ret_metabase('MOROCCO', "<query_id>"),
    # ret_metabase('MOROCCO', "<query_id>")
]

for df in Ecom_dfs:
    df.columns = map(str.lower, df.columns)

/home/ec2-user/service_account_key.json
/home/ec2-user/service_account_key.json
/home/ec2-user/service_account_key.json


In [15]:
# Call the Fintech Runsheet to check for retailers already assignied 
# if not data_instance_df.empty:
data_instance = google_sheets('Fintech Runsheet', 'Data', 'get')
progress      = google_sheets('Fintech Runsheet', 'Dashboard', 'get')
data_instance_df = data_instance.drop(columns=['Unnamed: 0'])
data_instance_df.columns = data_instance_df.iloc[0]
data_instance_df = data_instance_df[1:].reset_index(drop=True)
data_instance_df = data_instance_df.dropna(subset=['Retailer ID']).reset_index(drop=True)

# Convert to set of already assigned retailer IDs
already_assigned_retailers = set(data_instance_df['Retailer ID'].astype(int).unique())

# simple math
correct_table = progress.iloc[4:9, 7:9]  # Rows 6-9 (index 5-8), Columns G-N (index 6-12)
correct_table.columns = ["Total Assigned", "Total Calls Done"]

Tasks = correct_table["Total Assigned"].sum()
Done  = correct_table["Total Calls Done"].sum()

if Done > 0:
    progress_perc = round((Done / Tasks) * 100, 1)
    print(f"percentage of taks done {progress_perc}%")

/home/ec2-user/service_account_key.json
/home/ec2-user/service_account_key.json
percentage of taks done 82.3%


In [26]:
df_Retention

,retailer_id,name,mobile,sales_order_id,created_at,project_name,description,last_call_date,balance,priority


In [28]:
df_Retention = pd.concat(Retention_dfs, ignore_index=True)
df_Retention = df_Retention[~df_Retention['retailer_id'].isin(already_assigned_retailers)]
data_assigned = assign_data_equal_projects(df_Retention, available_agents_list)

In [30]:
if data_assigned.empty:
    print("youre good")
else:
    print("no")

youre good


In [43]:
# from airflow import DAG
# from airflow.operators.python import PythonOperator
from common_functions import ret_metabase, google_sheets, send_text_slack
from datetime import datetime, timedelta
import pandas as pd

    

def assign_data_equal_projects(df, list):
    df = df.sample(frac=1)  # Shuffle the data
    project_types = df['project_name'].unique()
    
    assigned_data = pd.DataFrame()
    
    for project in project_types:
        project_df = df[df['project_name'] == project]
        project_df = project_df.reset_index(drop=True)
        rows_per_agent = len(project_df) // len(list)
        remainder = len(project_df) % len(list)
        
        # Distribute rows equally
        for i, agent in enumerate(list):
            start_idx = i * rows_per_agent
            end_idx = start_idx + rows_per_agent
            agent_data = project_df.iloc[start_idx:end_idx].copy()
            agent_data['agent_assigned'] = agent
            
            # Handle remainder
            if i < remainder:
                extra_row = project_df.iloc[end_idx:end_idx+1].copy()
                extra_row['agent_assigned'] = agent
                agent_data = pd.concat([agent_data, extra_row])
            
            assigned_data = pd.concat([assigned_data, agent_data])
    
    assigned_data = assigned_data.reset_index(drop=True)
    
    return assigned_data

def assign_data_by_mapping(df, mapping_df):
    # store retail-agent mapping in a dictionary 
    mapping_dict = dict(zip(mapping_df['main_system_id'], mapping_df['agent_assigned']))
    
    
    assigned_agents = []
    # missing_retailers = [] could use it later for viewing errors

    for retailer_id in df['retailer_id']:
        if retailer_id in mapping_dict:
            assigned_agents.append(mapping_dict[retailer_id])
        else:
            assigned_agents.append(None)
    
    df['agent_assigned'] = assigned_agents

    df = df.reset_index(drop=True)

    return df

            

import pytz
from datetime import datetime
import pandas as pd
import numpy as np

morocco_tz = pytz.timezone('Africa/Casablanca')
now = datetime.now(morocco_tz)
current_time = now.time()
current_day = now.strftime('%A')
print(current_day)
hour = int(str(now.time())[0:2])

agents_df = google_sheets('MA daily_automatic_assignment', 'attendance_filtered', 'get')
# print(agents_df)
print(hour)

agents_df_copy = agents_df.copy() 
agents_df.columns = agents_df.columns.str.strip().str.lower()
agents_df_copy['assignment_start_time'] = agents_df['start_time'].astype(int)
agents_df_copy['assignment_end_time'] = agents_df['end_time'].astype(int)
agents_df_copy['current_day'] = agents_df['day'].astype(str).str.lower() == 'true'

# Fetch data for mapped retailer-agent 
mapping_df = google_sheets('MA Telesales Cycle Assignment', 'Sheet1', 'get')

# print(mapping_df['agent_assigned'])
agents_df_copy['assign_data'] =  np.where(
    (hour >= agents_df_copy['assignment_start_time'])
    & (hour <= agents_df_copy['assignment_end_time'])
    & (agents_df['day'] == True),
    'yes',
    'no' )

# store all available clocked-in agents for today
available_agents = agents_df_copy.loc[agents_df_copy['assign_data'] == 'yes']
available_agents_list = available_agents['agent_id'].values.tolist()
number_of_available_agents = len(available_agents_list)
# print(available_agents['agent_id'])
if number_of_available_agents != 0:


#     # Retention & Reactivation Tasks for 1st time slot
#     Retention_dfs = [
#         ret_metabase('MOROCCO', 18879),
#         # ret_metabase('MOROCCO', "<query_id>"),
#         # ret_metabase('MOROCCO', "<query_id>"),
#         # ret_metabase('MOROCCO', "<query_id>"),
#         # ret_metabase('MOROCCO', "<query_id>")
#     ]

#     for df in Retention_dfs:
#         df.columns = map(str.lower, df.columns)

#     # Marketing Tasks for 2nd time slot
#     Marketing_dfs = [
#         ret_metabase('MOROCCO', 10368),
#         # ret_metabase('MOROCCO', "<query_id>"),
#         # ret_metabase('MOROCCO', "<query_id>"),
#         # ret_metabase('MOROCCO', "<query_id>"),
#         # ret_metabase('MOROCCO', "<query_id>")
#     ]

#     for df in Marketing_dfs:
#         df.columns = map(str.lower, df.columns)

    # Marketing & E-commerce  Tasks for 3rd time slot 
    Ecom_dfs = [
        ret_metabase('MOROCCO', 10728),
        # ret_metabase('MOROCCO', "<query_id>"),
        # ret_metabase('MOROCCO', "<query_id>"),
        # ret_metabase('MOROCCO', "<query_id>"),
        # ret_metabase('MOROCCO', "<query_id>")
    ]

    for df in Ecom_dfs:
        df.columns = map(str.lower, df.columns)


    # Call the Fintech Runsheet to check for retailers already assignied 
    # if not data_instance_df.empty:
    data_instance = google_sheets('Fintech Runsheet', 'Data', 'get')
    progress = google_sheets('Fintech Runsheet', 'Dashboard', 'get')
    data_instance_df = data_instance.drop(columns=['Unnamed: 0'])
    data_instance_df.columns = data_instance_df.iloc[0]
    data_instance_df = data_instance_df[1:].reset_index(drop=True)
    data_instance_df = data_instance_df.dropna(subset=['Retailer ID']).reset_index(drop=True)

    # Convert to set of already assigned retailer IDs
    already_assigned_retailers = set(data_instance_df['Retailer ID'].astype(int).unique())


    
    # some math
    correct_table = progress.iloc[4:9, 7:9]  # Rows 6-9 (index 5-8), Columns G-N (index 6-12)
    correct_table.columns = ["Total Assigned", "Total Calls Done"]

    Tasks = correct_table["Total Assigned"].sum()
    Done  = correct_table["Total Calls Done"].sum()
    print(Tasks)
    print(Done)
    if Done > 0:
        progress_perc= (Done/Tasks) * 100
        print(progress_perc)
        
    data_assigned = pd.DataFrame()
        
    if hour < 12:
        # Retention & Reactivation Tasks for 1st time slot
        Retention_dfs = [ret_metabase('MOROCCO', 18879)]
        for df in Retention_dfs:
            df.columns = map(str.lower, df.columns)
            
        df_Retention = pd.concat(Retention_dfs, ignore_index=True)
        df_Retention = df_Retention[~df_Retention['retailer_id'].isin(already_assigned_retailers)]
        retention_assigned = assign_data_equal_projects(df_Retention, available_agents_list)
        data_assigned = pd.concat([data_assigned, retention_assigned], ignore_index=True)

    if hour >= 13 or progress_perc > 70:
       # Marketing Tasks for 2nd time slot
        Marketing_dfs = [ret_metabase('MOROCCO', 10368)]
        for df in Marketing_dfs:
            df.columns = map(str.lower, df.columns)
            
        if hour < 15:
            df_Marketing = pd.concat(Marketing_dfs, ignore_index=True)
            df_Marketing = df_Marketing[~df_Marketing['retailer_id'].isin(already_assigned_retailers)]
            marketing_assigned = assign_data_equal_projects(df_Marketing, available_agents_list)
            data_assigned = pd.concat([data_assigned, marketing_assigned], ignore_index=True)
        
        if hour >= 15 or progress_perc > 90:
            # Marketing & E-commerce  Tasks for 3rd time slot 
            Ecom_dfs = [ret_metabase('MOROCCO', 10728)]
            for df in Ecom_dfs:
                df.columns = map(str.lower, df.columns)
        
            df_Marketing = pd.concat(Marketing_dfs, ignore_index=True)
            df_Ecom = pd.concat(Ecom_dfs, ignore_index=True)
            df_marketing_ecom = pd.concat([df_Marketing, df_Ecom], ignore_index=True)
            df_marketing_ecom = df_marketing_ecom[~df_marketing_ecom['retailer_id'].isin(already_assigned_retailers)]
            marketing_ecom_assigned = assign_data_equal_projects(df_marketing_ecom, available_agents_list)
            data_assigned = pd.concat([data_assigned, marketing_ecom_assigned], ignore_index=True)

        if data_assigned.empty:
            print("No Data to assign")
            return 
            

    # test = ret_metabase('MOROCCO', 18879)
#     test2 = ret_metabase('MOROCCO', 10368)
#     test3 =ret_metabase('MOROCCO', 10728)
    
    # print("10410: \n", test.columns.tolist())
#     print("10368: \n", test2.columns.tolist())
#     print("10728: \n", test3.columns.tolist())
#     print(data_assigned.columns.tolist())

    df_filtered = data_assigned.drop_duplicates(subset=['retailer_id'], keep="first")
    if not df_filtered.empty:
        df_filtered['retailer_id'] = df_filtered['retailer_id'].replace(',', '', regex=True).astype('int')
    else:
        print("df_filtered is empty, skipping replacement")
    df_filtered = df_filtered.reset_index(drop=True)
    print(df_filtered)



    final_data_to_assign = df_filtered.copy()
    final_data_to_assign['added_at'] = now.strftime('%Y-%m-%d %H:%M')
    # sheet_df = final_data_to_assign[["main_system_id", "retailer_mobile_number","retailer_name","description","reward","balance","offer","agent_assigned","added_at"]]
    if final_data_to_assign.columns[0] == 'index':
        final_data_to_assign = final_data_to_assign.drop(labels='index', axis=1)
    pass
    # Sort by 'priority' in ascending order
    if "priority" in final_data_to_assign.columns:
        final_data_to_assign = final_data_to_assign.sort_values(by='priority', ascending=True)
    
    final_data_to_assign = final_data_to_assign.astype(str)
    if "priority" in final_data_to_assign.columns:
        final_data_to_assign = final_data_to_assign[['retailer_id', 'name', 'mobile', 'sales_order_id', 'created_at', 'project_name', 'description', 'last_call_date', 'balance', 'agent_assigned', 'added_at', 'priority']]

    # google_sheets('Fintech Runsheet', 'raw_data', 'append', df=final_data_to_assign)
    
    print(final_data_to_assign)

else:
    print("no agents available")



Monday
/home/ec2-user/service_account_key.json
12
/home/ec2-user/service_account_key.json
/home/ec2-user/service_account_key.json
/home/ec2-user/service_account_key.json
/home/ec2-user/service_account_key.json
379
303
79.94722955145119
/home/ec2-user/service_account_key.json
/home/ec2-user/service_account_key.json
     retailer_id         name     mobile sales_order_id created_at  \
0          45395       rachid  664394910              0              
1          69497         لحسن  619676309              0              
2          19529  البصري سعيد  639941444              0              
3          56577        salah  720191020              0              
4          65759         محمد  783466966              0              
..           ...          ...        ...            ...        ...   
233        70244       بوجمعة  780480993              0              
234        38575         ياسر  696523872              0              
235        10665      mohamad  639391825              

/tmp/ipykernel_31237/1036716469.py:220: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['retailer_id'] = df_filtered['retailer_id'].replace(',', '', regex=True).astype('int')


In [41]:
df_Retention

,retailer_id,name,mobile,sales_order_id,created_at,project_name,description,last_call_date,balance,priority


In [48]:
def morocco_account_mgmt_dispatching():
    import pytz
    from datetime import datetime
    import pandas as pd
    import numpy as np

    morocco_tz = pytz.timezone('Africa/Casablanca')
    now = datetime.now(morocco_tz)
    current_time = now.time()
    current_day = now.strftime('%A')
    print(current_day)
    hour = int(str(now.time())[0:2])

    agents_df = google_sheets('MA daily_automatic_assignment', 'attendance_filtered', 'get')
    # print(agents_df)
    print(hour)

    agents_df_copy = agents_df.copy() 
    agents_df.columns = agents_df.columns.str.strip().str.lower()
    agents_df_copy['assignment_start_time'] = agents_df['start_time'].astype(int)
    agents_df_copy['assignment_end_time'] = agents_df['end_time'].astype(int)
    agents_df_copy['current_day'] = agents_df['day'].astype(str).str.lower() == 'true'


    # Fetch data for mapped retailer-agent 
    mapping_df = google_sheets('MA Telesales Cycle Assignment', 'Sheet1', 'get')


    agents_df_copy['assign_data'] =  np.where(
        (hour >= agents_df_copy['assignment_start_time'])
        & (hour <= agents_df_copy['assignment_end_time'])
        & (agents_df['day'] == True),
        'yes',
        'no' )

    # store all available clocked-in agents for today
    available_agents = agents_df_copy.loc[agents_df_copy['assign_data'] == 'yes']
    available_agents_list = available_agents['agent_id'].values.tolist()
    number_of_available_agents = len(available_agents_list)

    if number_of_available_agents != 0:

        # Call the Fintech Runsheet to check for retailers already assignied 
        # if not data_instance_df.empty:
        data_instance = google_sheets('Fintech Runsheet', 'Data', 'get')
        progress      = google_sheets('Fintech Runsheet', 'Dashboard', 'get')
        data_instance_df = data_instance.drop(columns=['Unnamed: 0'])
        data_instance_df.columns = data_instance_df.iloc[0]
        data_instance_df = data_instance_df[1:].reset_index(drop=True)
        data_instance_df = data_instance_df.dropna(subset=['Retailer ID']).reset_index(drop=True)

        # Convert to set of already assigned retailer IDs
        already_assigned_retailers = set(data_instance_df['Retailer ID'].astype(int).unique())
        
        # simple math
        correct_table = progress.iloc[4:9, 7:9]  # Rows 6-9 (index 5-8), Columns G-N (index 6-12)
        correct_table.columns = ["Total Assigned", "Total Calls Done"]

        Tasks = correct_table["Total Assigned"].sum()
        Done  = correct_table["Total Calls Done"].sum()
        print(f"Total Tasks: {Tasks}")
        print(f"Tasks Done: {Done}")
        if Done > 0:
            progress_perc = round((Done / Tasks) * 100, 1)
            print(f"percentage of taks done {progress_perc}%")

        data_assigned = pd.DataFrame()
        
        if hour < 12:
            # Retention & Reactivation Tasks for 1st time slot
            Retention_dfs = [ret_metabase('MOROCCO', 18879)]
            for df in Retention_dfs:
                df.columns = map(str.lower, df.columns)
                
            df_Retention = pd.concat(Retention_dfs, ignore_index=True)
            df_Retention = df_Retention[~df_Retention['retailer_id'].isin(already_assigned_retailers)]
            retention_assigned = assign_data_equal_projects(df_Retention, available_agents_list)
            data_assigned = pd.concat([data_assigned, retention_assigned], ignore_index=True)

            if not retention_assigned.empty:
                print(f"{len(retention_assigned)} data available in Retention")

        if hour >= 13 or progress_perc > 70:
        # Marketing Tasks for 2nd time slot
            Marketing_dfs = [ret_metabase('MOROCCO', 10368)]
            for df in Marketing_dfs:
                df.columns = map(str.lower, df.columns)
                
            if hour < 15 and progress_perc < 90:
                df_Marketing = pd.concat(Marketing_dfs, ignore_index=True)
                df_Marketing = df_Marketing[~df_Marketing['retailer_id'].isin(already_assigned_retailers)]
                marketing_assigned = assign_data_equal_projects(df_Marketing, available_agents_list)
                data_assigned = pd.concat([data_assigned, marketing_assigned], ignore_index=True)
                if not marketing_assigned.empty:
                    print(f"{len(marketing_assigned)} data available in Marketing")
            
            if hour >= 15 or progress_perc >= 90:
                # Marketing & E-commerce  Tasks for 3rd time slot 
                Ecom_dfs = [ret_metabase('MOROCCO', 10728)]
                for df in Ecom_dfs:
                    df.columns = map(str.lower, df.columns)
            
                df_Marketing = pd.concat(Marketing_dfs, ignore_index=True)
                df_Ecom = pd.concat(Ecom_dfs, ignore_index=True)
                df_marketing_ecom = pd.concat([df_Marketing, df_Ecom], ignore_index=True)
                df_marketing_ecom = df_marketing_ecom[~df_marketing_ecom['retailer_id'].isin(already_assigned_retailers)]
                marketing_ecom_assigned = assign_data_equal_projects(df_marketing_ecom, available_agents_list)
                data_assigned = pd.concat([data_assigned, marketing_ecom_assigned], ignore_index=True)
                if not marketing_ecom_assigned.empty:
                    print(f"{len(marketing_ecom_assigned)} data available in Marketing and E-com")

            if data_assigned.empty:
                print("No Data to assign")
                return        

        # Concatenate the dataframes along the rows (union all)
        # df_unfiltered = pd.concat([data_asigned,recharge_data_assigned], ignore_index=True)
        df_filtered = data_assigned.drop_duplicates(subset=['retailer_id'], keep="first")
        if not df_filtered.empty:
            df_filtered['retailer_id'] = df_filtered['retailer_id'].replace(',', '', regex=True).astype('int')
        else:
            print("df_filtered is empty, skipping replacement")
        df_filtered = df_filtered.reset_index(drop=True)
        # print(df_filtered)


        #Dispatching data to agents' sheet 
        final_data_to_assign = df_filtered.copy()
        final_data_to_assign['added_at'] = now.strftime('%Y-%m-%d %H:%M')
        if final_data_to_assign.columns[0] == 'index':
            final_data_to_assign = final_data_to_assign.drop(labels='index', axis=1)
        pass
        
        # Sort by 'priority' in ascending order
        if "priority" in final_data_to_assign.columns:
            final_data_to_assign = final_data_to_assign.sort_values(by='priority', ascending=True)

        final_data_to_assign = final_data_to_assign.astype(str)

        if "priority" in final_data_to_assign.columns:
            final_data_to_assign = final_data_to_assign[['retailer_id', 'name', 'mobile', 'sales_order_id', 'created_at', 'project_name', 'description', 'last_call_date', 'balance', 'agent_assigned', 'added_at', 'priority']]
        print(final_data_to_assign)
       
        # google_sheets('Fintech Runsheet', 'raw_data', 'append', df=final_data_to_assign)

#         file_name = f'''{now.strftime("%Y")}/{now.strftime("%m")}/{now.strftime("%d")}/account_mgmt_dispatched_data'''

#         try:
#             hist_data_from_s3 = pd.read_excel(f'''s3://snowflake-maxabdwh-morocco/dwh-initial-load/fintech/account_mgmt_dispatching/{file_name}.xlsx''')
#             hist_data_from_s3 = pd.concat([hist_data_from_s3, final_data_to_assign], ignore_index=True)
#         except Exception as e:
#             print(e)
#             hist_data_from_s3 = final_data_to_assign.copy()

#         hist_data_from_s3.to_excel(
#             f'''s3://snowflake-maxabdwh-morocco/dwh-initial-load/fintech/account_mgmt_dispatching/{file_name}.xlsx''',
#             index=False)
    else:
        print("no agents available")

In [50]:
morocco_account_mgmt_dispatching()

Monday
/home/ec2-user/service_account_key.json
14
/home/ec2-user/service_account_key.json
/home/ec2-user/service_account_key.json
/home/ec2-user/service_account_key.json
Total Tasks: 550
Tasks Done: 375
percentage of taks done 68.2%
/home/ec2-user/service_account_key.json
1 data available in Marketing
  retailer_id   name     mobile sales_order_id  created_at  \
0       74609  سعيد   784166355                 2025-06-02   

            project_name                                        description  \
0  marketing_acquisition  Digital Acquisition + District(si existe): 31....   

  last_call_date balance agent_assigned          added_at priority  
0            nan       0          128.0  2025-06-02 14:21        2  


In [4]:
from common_functions import google_sheets

In [5]:
progress      = google_sheets('Fintech Runsheet', 'Dashboard', 'get')

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


/home/ec2-user/service_account_key.json


In [7]:
correct_table = progress.iloc[4:9, 7:9]  # Rows 6-9 (index 5-8), Columns G-N (index 6-12)
correct_table.columns = ["Total Assigned", "Total Calls Done"]
correct_table

,Total Assigned,Total Calls Done
6,98,91
7,102,94
8,98,91
9,98,61
10,98,24


In [8]:
# Sort by "Total Calls Done" in descending order and get the top 3
top_3_done = correct_table.sort_values(by="Total Calls Done", ascending=False).head(3)
top_3_done


,Total Assigned,Total Calls Done
7,102,94
6,98,91
8,98,91


In [10]:
Tasks = top_3_done["Total Assigned"].sum()
Done  = top_3_done["Total Calls Done"].sum()
print(f"Total Tasks: {Tasks}")
print(f"Tasks Done: {Done}")

Total Tasks: 298
Tasks Done: 276


In [11]:
        progress_perc = 0
        if Done > 0:
            progress_perc = round((Done / Tasks) * 100, 1)
            print(f"percentage of taks done {progress_perc}%")

percentage of taks done 92.6%
